In [3]:
import xlsxwriter
from datetime import datetime

# Crear el archivo de Excel
filename = 'plantilla_progreso.xlsx'
workbook = xlsxwriter.Workbook(filename)

# Crear las dos hojas necesarias
ws_progreso = workbook.add_worksheet('progreso')
ws_config = workbook.add_worksheet('config') # Esta hoja tendrá la lista de productos

# ---------------------------------------------------------
# 1. CONFIGURAR LA HOJA 'config' (Lista de Productos)
# ---------------------------------------------------------
ws_config.write('A1', 'LISTA DE PRODUCTOS')
# Agregamos algunos productos de ejemplo
productos_ejemplo = ['Producto 1', 'Producto 2', 'Producto 3']
for i, prod in enumerate(productos_ejemplo):
    ws_config.write(i + 1, 0, prod)

# Instrucción visual para el usuario
ws_config.write('B2', '<- Escribe nuevos productos en esta columna A')

# ---------------------------------------------------------
# 2. CONFIGURAR LA HOJA 'progreso'
# ---------------------------------------------------------
headers = ['ID', 'Date', 'Product', 'Tons', 'Time']
header_format = workbook.add_format({'bold': True, 'bg_color': '#D3D3D3', 'border': 1})

# Escribir encabezados
for col_num, header in enumerate(headers):
    ws_progreso.write(0, col_num, header, header_format)
    ws_progreso.set_column(col_num, col_num, 15) # Ajustar ancho de columna

# Definimos el rango donde aplicarán las reglas (ej. hasta la fila 1000)
row_start = 1
row_end = 1000

# --- REGLA 1: PRODUCT (Columna C) ---
# Debe ser una lista tomada de la hoja 'config'.
# Usamos un rango amplio ($A$2:$A$500) para que el usuario pueda agregar más productos después.
ws_progreso.data_validation(row_start, 2, row_end, 2, {
    'validate': 'list',
    'source': '=config!$A$2:$A$500',
    'input_title': 'Seleccionar Producto',
    'input_message': 'Elige un producto de la lista.'
})

# --- REGLA 2: TONS (Columna D) ---
# Solo números entre 0 y 40.
ws_progreso.data_validation(row_start, 3, row_end, 3, {
    'validate': 'decimal',
    'criteria': 'between',
    'minimum': 0,
    'maximum': 40,
    'error_title': 'Error de peso',
    'error_message': 'El valor debe estar entre 0 y 40 toneladas.'
})

# --- REGLA 3: TIME (Columna E) ---
# Debe ser una hora válida.
time_format = workbook.add_format({'num_format': 'hh:mm'})
ws_progreso.set_column(4, 4, 15, time_format) # Aplicar formato visual HH:MM
ws_progreso.data_validation(row_start, 4, row_end, 4, {
    'validate': 'time',
    'criteria': 'between',
    'minimum': 0,     # 00:00
    'maximum': 0.99999, # 23:59:59
    'error_title': 'Hora inválida',
    'error_message': 'Por favor ingresa una hora válida (HH:MM).'
})

# --- REGLA 4: DATE (Columna B) ---
# Fecha menor o igual al día actual.
date_format = workbook.add_format({'num_format': 'dd/mm/yyyy'})
ws_progreso.set_column(1, 1, 15, date_format)

# Nota: La validación dinámica "=HOY()" a veces requiere ajustes al importar a Sheets,
# pero configuramos la base aquí.
ws_progreso.data_validation(row_start, 1, row_end, 1, {
    'validate': 'date',
    'criteria': '<=',
    'value': '=TODAY()',
    'error_title': 'Fecha futura',
    'error_message': 'La fecha no puede ser mayor al día de hoy.'
})

workbook.close()
print(f"Archivo generado: {filename}")

Archivo generado: plantilla_progreso.xlsx


In [5]:
import xlsxwriter

# Nombre del archivo
filename = 'Plantilla_Sackoff.xlsx'
workbook = xlsxwriter.Workbook(filename)

# Crear las dos hojas
ws_main = workbook.add_worksheet('Seguimiento Sackoff')
ws_config = workbook.add_worksheet('Config')

# ============================================================================
# 1. CONFIGURACIÓN DE LA HOJA 'Config' (Listas y Límites)
# ============================================================================
# Encabezados
config_headers = ['Lista Productos', 'Lista Pellets', 'Lista Operarios', 'Lista Silos', '', 'CONFIGURACIÓN LÍMITES', 'Valor']
header_fmt_conf = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0', 'border': 1})
ws_config.write_row('A1', config_headers, header_fmt_conf)

# Datos de prueba para las listas (Relleno inicial)
ws_config.write_column('A2', ['Producto A', 'Producto B', 'Producto C', 'Producto D'])
ws_config.write_column('B2', ['Pellet X', 'Pellet Y', 'Pellet Z'])
ws_config.write_column('C2', ['Operario 1', 'Operario 2', 'Operario 3'])
ws_config.write_column('D2', ['Silo 1', 'Silo 2', 'Silo 3'])

# Configuración de límites de Toneladas (Min y Max)
ws_config.write('F2', 'Min Tons')
ws_config.write('G2', 0)    # Límite Inferior (Configurable)
ws_config.write('F3', 'Max Tons')
ws_config.write('G3', 200)  # Límite Superior (Configurable)

ws_config.write('F5', 'NOTA: Modifica G2 y G3 para cambiar los límites en la hoja principal.')

# ============================================================================
# 2. CONFIGURACIÓN DE LA HOJA 'Seguimiento Sackoff'
# ============================================================================

# Encabezados solicitados
headers = [
    'id',               # A - Auto (UUID)
    'fecha_id',         # B - Auto (Timestamp sistema)
    'fecha',            # C - Operario (Validación <= Hoy)
    'lote',             # D - Numérico (Obligatorio)
    'producto',         # E - Lista
    'tons_teorico',     # F - Numérico (Límites)
    'tons_real',        # G - Numérico (Límites)
    'tons_real_vascula',# H - Numérico (Límites)
    'pellet',           # I - Lista
    'operario',         # J - Lista
    'ano',              # K - Calculado
    'mes',              # L - Calculado
    'silo'              # M - Lista
]

# Estilos
header_fmt_main = workbook.add_format({'bold': True, 'bg_color': '#4A86E8', 'font_color': 'white', 'border': 1, 'align': 'center'})
locked_fmt = workbook.add_format({'bg_color': '#EFEFEF', 'italic': True, 'font_color': '#666666'}) # Para columnas auto
date_fmt = workbook.add_format({'num_format': 'dd/mm/yyyy'})
alert_fmt = workbook.add_format({'bg_color': '#FFCCCC', 'font_color': '#990000', 'bold': True}) # Alerta Lote

# Escribir encabezados
ws_main.write_row('A1', headers, header_fmt_main)

# Ajustar ancho de columnas para mejor visualización
ws_main.set_column('A:B', 20) # IDs
ws_main.set_column('C:C', 12) # Fecha
ws_main.set_column('E:E', 15) # Producto
ws_main.set_column('F:H', 15) # Tons
ws_main.set_column('I:M', 12) # Resto

# Definir rango de filas operativas (ej. de la 2 a la 2000)
row_start = 1
row_end = 2000

# ---------------- VALIDACIONES Y REGLAS ----------------

# 1. FECHA (Columna C): No mayor a hoy + Mensaje de ayuda
ws_main.data_validation(row_start, 2, row_end, 2, {
    'validate': 'date',
    'criteria': '<=',
    'value': '=TODAY()',
    'input_title': 'Formato Fecha',
    'input_message': 'Ingresa la fecha (DD/MM/AAAA).',
    'error_title': 'Fecha Inválida',
    'error_message': 'La fecha no puede ser futura ni tener un formato incorrecto.'
})

# 2. LOTE (Columna D): Numérico + Formato condicional si está vacío y hay fecha
ws_main.data_validation(row_start, 3, row_end, 3, {
    'validate': 'decimal',
    'criteria': '>=',
    'value': 0
})
# Si C (Fecha) tiene dato Y D (Lote) está vacío -> PINTAR ROJO
ws_main.conditional_format(row_start, 3, row_end, 3, {
    'type': 'formula',
    'criteria': '=AND($C2<>"", $D2="")',
    'format': alert_fmt
})

# 3. PRODUCTO (Col E), PELLET (Col I), OPERARIO (Col J), SILO (Col M): Listas desplegables
# Referencias a la hoja Config (usamos un rango amplio $2:$100 para permitir agregar más)
ws_main.data_validation(row_start, 4, row_end, 4, {'validate': 'list', 'source': '=Config!$A$2:$A$100'}) # Producto
ws_main.data_validation(row_start, 8, row_end, 8, {'validate': 'list', 'source': '=Config!$B$2:$B$100'}) # Pellet
ws_main.data_validation(row_start, 9, row_end, 9, {'validate': 'list', 'source': '=Config!$C$2:$C$100'}) # Operario
ws_main.data_validation(row_start, 12, row_end, 12, {'validate': 'list', 'source': '=Config!$D$2:$D$100'}) # Silo

# 4. TONS (Cols F, G, H): Límites dinámicos desde Config
for col_idx in [5, 6, 7]: # Índices 5, 6, 7 corresponden a F, G, H
    ws_main.data_validation(row_start, col_idx, row_end, col_idx, {
        'validate': 'decimal',
        'criteria': 'between',
        'minimum': '=Config!$G$2', # Min Tons
        'maximum': '=Config!$G$3', # Max Tons
        'error_title': 'Fuera de rango',
        'error_message': 'El valor excede los límites configurados en la hoja Config.'
    })

# 5. AÑO y MES (Cols K, L): Fórmulas automáticas
# Escribimos la fórmula en la primera fila de datos como referencia
ws_main.write_formula('K2', '=IF(C2<>"", YEAR(C2), "")')
ws_main.write_formula('L2', '=IF(C2<>"", MONTH(C2), "")')

# Cerrar y guardar
workbook.close()
print(f"Archivo '{filename}' generado exitosamente.")

Archivo 'Plantilla_Sackoff.xlsx' generado exitosamente.


In [6]:
import xlsxwriter

filename = 'Plantilla_Sackoff_V2.xlsx'
workbook = xlsxwriter.Workbook(filename)

# Crear hojas
ws_main = workbook.add_worksheet('Seguimiento Sackoff')
ws_config = workbook.add_worksheet('Config')

# ============================================================================
# 1. CONFIGURACIÓN (Listas y Tolerancias Sackoff)
# ============================================================================
# Encabezados
config_headers = ['Lista Productos', 'Lista Pellets', 'Lista Operarios', 'Lista Silos', '', 'CONFIG LIMITES TONS', 'Valor']
header_fmt_conf = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0', 'border': 1})
ws_config.write_row('A1', config_headers, header_fmt_conf)

# Datos de relleno
ws_config.write_column('A2', ['Producto A', 'Producto B', 'Producto C'])
ws_config.write_column('B2', ['Pellet X', 'Pellet Y'])
ws_config.write_column('C2', ['Operario 1', 'Operario 2'])
ws_config.write_column('D2', ['Silo 1', 'Silo 2'])

# Límites Toneladas
ws_config.write('F2', 'Min Tons')
ws_config.write('G2', 0)
ws_config.write('F3', 'Max Tons')
ws_config.write('G3', 200)

# --- NUEVO: Configuración de Sackoff ---
ws_config.write('F5', 'SACKOFF TOLERANCIA')
ws_config.write('F6', 'Min (%)')
ws_config.write('G6', -1.5)  # Límite Inferior
ws_config.write('F7', 'Max (%)')
ws_config.write('G7', 1.5)   # Límite Superior

ws_config.write('F9', 'Instrucción:', workbook.add_format({'bold':True}))
ws_config.write('F10', 'Si el % está entre G6 y G7 será VERDE, sino ROJO.')

# ============================================================================
# 2. HOJA PRINCIPAL
# ============================================================================

headers = [
    'ID',                # A
    'Fecha ID',          # B
    'Fecha',             # C
    'Lote',              # D
    'Producto',          # E
    'Tons Teorico',      # F
    'Tons Real',         # G
    'Tons Real Báscula', # H
    'Sackoff %',         # I (NUEVA COLUMNA DE CÁLCULO)
    'Pellet',            # J
    'Operario',          # K
    'Año',               # L
    'Mes',               # M
    'Silo'               # N
]

# Estilos
header_fmt_main = workbook.add_format({'bold': True, 'bg_color': '#4A86E8', 'font_color': 'white', 'border': 1, 'align': 'center'})
# Formatos de alerta
red_format = workbook.add_format({'bg_color': '#FF9999', 'font_color': '#9C0006'}) # Rojo para fuera de rango
green_format = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'}) # Verde para OK
# Formato número
decimal_fmt = workbook.add_format({'num_format': '0.00'})

# Escribir encabezados
ws_main.write_row('A1', headers, header_fmt_main)

# Ajustar anchos
ws_main.set_column('A:B', 22) # ID y Fecha ID visibles
ws_main.set_column('C:C', 12)
ws_main.set_column('D:E', 15)
ws_main.set_column('F:H', 15)
ws_main.set_column('I:I', 15) # Columna Sackoff

row_start = 1
row_end = 2000

# --- VALIDACIONES BÁSICAS (Igual que antes) ---
ws_main.data_validation(row_start, 2, row_end, 2, {'validate': 'date', 'criteria': '<=', 'value': '=TODAY()'}) # Fecha
ws_main.data_validation(row_start, 3, row_end, 3, {'validate': 'decimal', 'criteria': '>=', 'value': 0}) # Lote
ws_main.conditional_format(row_start, 3, row_end, 3, {'type': 'formula', 'criteria': '=AND($C2<>"", $D2="")', 'format': red_format}) # Alerta Lote

# Listas
ws_main.data_validation(row_start, 4, row_end, 4, {'validate': 'list', 'source': '=Config!$A$2:$A$100'}) # Producto
ws_main.data_validation(row_start, 9, row_end, 9, {'validate': 'list', 'source': '=Config!$B$2:$B$100'}) # Pellet (Ahora col J -> index 9)
ws_main.data_validation(row_start, 10, row_end, 10, {'validate': 'list', 'source': '=Config!$C$2:$C$100'}) # Operario
ws_main.data_validation(row_start, 13, row_end, 13, {'validate': 'list', 'source': '=Config!$D$2:$D$100'}) # Silo

# Límites Tons (Cols F, G, H -> indices 5, 6, 7)
for col_idx in [5, 6, 7]:
    ws_main.data_validation(row_start, col_idx, row_end, col_idx, {
        'validate': 'decimal', 'criteria': 'between', 
        'minimum': '=Config!$G$2', 'maximum': '=Config!$G$3'
    })

# --- NUEVO: FÓRMULA SACKOFF Y COLORES ---
# Columna I (índice 8). Fórmula: (Real - Teorico) / Real * 100
# Agregamos IF para evitar división por cero si Real está vacío o es 0.
# Excel: =SI(G2<>0, (G2-F2)/G2*100, "")
ws_main.write_formula('I2', '=IF(G2<>0, (G2-F2)/G2*100, "")', decimal_fmt)

# Formato Condicional SACKOFF
# 1. VERDE: Si está entre Min y Max (inclusive)
# Formula: Y(Valor >= Min, Valor <= Max)
ws_main.conditional_format(row_start, 8, row_end, 8, {
    'type': 'formula',
    'criteria': '=AND($I2 >= Config!$G$6, $I2 <= Config!$G$7, $I2<>"")',
    'format': green_format
})

# 2. ROJO: Si está fuera del rango (Menor que Min O Mayor que Max)
# Formula: O(Valor < Min, Valor > Max)
ws_main.conditional_format(row_start, 8, row_end, 8, {
    'type': 'formula',
    'criteria': '=OR($I2 < Config!$G$6, $I2 > Config!$G$7)',
    'format': red_format
})

# Formulas Año/Mes (Ahora cols L, M -> indices 11, 12)
ws_main.write_formula('L2', '=IF(C2<>"", YEAR(C2), "")')
ws_main.write_formula('M2', '=IF(C2<>"", MONTH(C2), "")')

workbook.close()
print("Archivo generado con regla de Sackoff.")

Archivo generado con regla de Sackoff.


In [7]:
import xlsxwriter

filename = 'Sistema_Sackoff_SinScripts.xlsx'
workbook = xlsxwriter.Workbook(filename)

# 1. Crear hojas
ws_instr = workbook.add_worksheet('LEER PRIMERO') # Hoja de instrucciones
ws_main = workbook.add_worksheet('Seguimiento Sackoff')
ws_config = workbook.add_worksheet('Config')

# ============================================================================
# HOJA 0: INSTRUCCIONES (CRUCIAL PARA QUE FUNCIONEN LAS FÓRMULAS SIN SCRIPT)
# ============================================================================
instr_fmt = workbook.add_format({'bold': True, 'font_size': 14, 'font_color': 'red'})
ws_instr.write('A1', '¡IMPORTANTE: CONFIGURACIÓN REQUERIDA!', instr_fmt)
ws_instr.write('A3', 'Para que los IDs y Fechas automáticas funcionen sin Scripts, haz esto:')
ws_instr.write('A5', 'EN GOOGLE SHEETS:')
ws_instr.write('A6', '1. Ve a Archivo > Configuración.')
ws_instr.write('A7', '2. Pestaña "Cálculo".')
ws_instr.write('A8', '3. "Cálculo iterativo" -> ACTIVADO.')
ws_instr.write('A9', '4. "Número máximo de iteraciones" -> Pon 1.')
ws_instr.write('A10', '5. Guardar configuración.')
ws_instr.write('A12', 'EN EXCEL (Escritorio):')
ws_instr.write('A13', '1. Archivo > Opciones > Fórmulas.')
ws_instr.write('A14', '2. Marcar "Habilitar cálculo iterativo".')
ws_instr.write('A15', '3. Iteraciones máximas: 1.')

# ============================================================================
# HOJA 2: CONFIGURACIÓN
# ============================================================================
header_fmt = workbook.add_format({'bold': True, 'bg_color': '#DDDDDD', 'border': 1})
ws_config.write_row('A1', ['Productos', 'Pellets', 'Operarios', 'Silos', '', 'LIMITES TONS', 'Valor'], header_fmt)

# Datos base
ws_config.write_column('A2', ['Producto A', 'Producto B', 'Producto C'])
ws_config.write_column('B2', ['Pellet 1', 'Pellet 2'])
ws_config.write_column('C2', ['Juan', 'Pedro', 'Maria'])
ws_config.write_column('D2', ['Silo A', 'Silo B'])

# Límites
ws_config.write('F2', 'Min Tons'); ws_config.write('G2', 0)
ws_config.write('F3', 'Max Tons'); ws_config.write('G3', 200)
ws_config.write('F5', 'Sackoff Min %'); ws_config.write('G5', -1.5)
ws_config.write('F6', 'Sackoff Max %'); ws_config.write('G6', 1.5)

# ============================================================================
# HOJA 1: SEGUIMIENTO SACKOFF
# ============================================================================
headers = [
    'ID (Auto)', 'Fecha Reg (Auto)', 'Fecha (Operario)', 'Lote', 'Producto',
    'Tons Teorico', 'Tons Real', 'Tons Real Báscula', 'Sackoff %',
    'Pellet', 'Operario', 'Año', 'Mes', 'Silo'
]
main_header_fmt = workbook.add_format({'bold': True, 'bg_color': '#4A86E8', 'font_color': 'white', 'border': 1})
ws_main.write_row('A1', headers, main_header_fmt)

# Formatos
date_fmt = workbook.add_format({'num_format': 'dd/mm/yyyy'})
datetime_fmt = workbook.add_format({'num_format': 'dd/mm/yyyy hh:mm'})
green_fmt = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
red_fmt = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
lock_fmt = workbook.add_format({'bg_color': '#F3F3F3', 'italic': True})

row_start = 1
row_end = 1000

# 1. FÓRMULAS MÁGICAS (REFERENCIA CIRCULAR) - SIN SCRIPTS
# ID (Col A): Si C tiene dato, y A está vacío, genera Aleatorio. Si A ya tiene dato, MANTENER A.
# Nota: Usamos fórmulas en inglés para compatibilidad universal al generar el xlsx
ws_main.write_formula('A2', '=IF(C2="","",IF(A2<>0,A2,RANDBETWEEN(100000,999999)))', lock_fmt)

# Fecha ID (Col B): Si C tiene dato, y B está vacío, pon AHORA. Si B tiene dato, MANTENER B.
ws_main.write_formula('B2', '=IF(C2="","",IF(B2<>0,B2,NOW()))', datetime_fmt)

# 2. Validaciones
# Fecha Operario (Col C) <= HOY
ws_main.data_validation(row_start, 2, row_end, 2, {'validate': 'date', 'criteria': '<=', 'value': '=TODAY()'})

# Lote (Col D) - Alerta visual si vacío
ws_main.data_validation(row_start, 3, row_end, 3, {'validate': 'decimal', 'criteria': '>=', 'value': 0})
ws_main.conditional_format(row_start, 3, row_end, 3, {'type': 'formula', 'criteria': '=AND($C2<>"", $D2="")', 'format': red_fmt})

# Listas (Producto, Pellet, Operario, Silo)
ws_main.data_validation(row_start, 4, row_end, 4, {'validate': 'list', 'source': '=Config!$A$2:$A$100'})
ws_main.data_validation(row_start, 9, row_end, 9, {'validate': 'list', 'source': '=Config!$B$2:$B$100'})
ws_main.data_validation(row_start, 10, row_end, 10, {'validate': 'list', 'source': '=Config!$C$2:$C$100'})
ws_main.data_validation(row_start, 13, row_end, 13, {'validate': 'list', 'source': '=Config!$D$2:$D$100'})

# Tons (F, G, H)
for col in [5, 6, 7]:
    ws_main.data_validation(row_start, col, row_end, col, {
        'validate': 'decimal', 'criteria': 'between', 
        'minimum': '=Config!$G$2', 'maximum': '=Config!$G$3'
    })

# 3. Sackoff (Col I) - Fórmula y Colores
ws_main.write_formula('I2', '=IF(G2<>0, (G2-F2)/G2*100, "")')
# Verde (Dentro del rango)
ws_main.conditional_format(row_start, 8, row_end, 8, {
    'type': 'formula', 
    'criteria': '=AND($I2>=Config!$G$5, $I2<=Config!$G$6, $I2<>"")', 
    'format': green_fmt
})
# Rojo (Fuera del rango)
ws_main.conditional_format(row_start, 8, row_end, 8, {
    'type': 'formula', 
    'criteria': '=OR($I2<Config!$G$5, $I2>Config!$G$6)', 
    'format': red_fmt
})

# 4. Año y Mes
ws_main.write_formula('L2', '=IF(C2<>"", YEAR(C2), "")')
ws_main.write_formula('M2', '=IF(C2<>"", MONTH(C2), "")')

workbook.close()
print("Archivo generado. Recuerda activar el CÁLCULO ITERATIVO en Excel/Sheets.")

Archivo generado. Recuerda activar el CÁLCULO ITERATIVO en Excel/Sheets.


In [ ]:
graph TD
    %% Estilos para nodos
    classDef input fill:#e1f5fe,stroke:#01579b,stroke-width:2px;
    classDef process fill:#fff3e0,stroke:#e65100,stroke-width:2px;
    classDef output fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px;
    classDef dataPoint fill:#ffeb3b,stroke:#fbc02d,stroke-width:2px,stroke-dasharray: 5 5;

    subgraph ENTRADAS_MACRO [PUNTO 1: ENTRADAS AL SISTEMA]
        A[Báscula Camionera<br>Materias Primas] --> B(Silos de Almacenaje)
        
    end

    B --> D{Molienda y<br>Dosificación}

    subgraph PUNTO_CRITICO_INICIO [PUNTO 2: EL CERO DEL PROCESO]
        D --> E[MEZCLADORA PRINCIPAL]
        E -- DATOS CLAVE --> DP2([DATA POINT 2:<br>- Peso Total a producir por día/lotes/op's<br>- % Humedad Harina Inicial por día/lotes/ops a producir])
    end

    E --> F[Tolva hacia el acondicionador<br>]

    subgraph ZONA_DE_TRANSFORMACION [PUNTO 3 y 4: LA CAJA NEGRA]
        F --> G[Acondicionador]
        H[Caldera / Vapor] -- GANANCIA DE PESO --> G
        G -- DATOS CLAVE --> DP3([DATA POINT 3:<br>- Flujo de Vapor kg/h por Lote/Ops/Día<br>- Temp. Acondicionamiento por Lote/Ops/Día])
        G --> I[Tambor con Dado]
        I --> J[ENFRIADOR / SECADOR]
        J -- PÉRDIDA DE PESO (Humedad/Calor) --> K[Atmósfera]
        K -- Recuperación Finos --> F
        J -- DATOS CLAVE --> DP4([DATA POINT 4:<br>- % Humedad Pellet Final por Lote/Ops/Día<br>- Temperatura Salida por día/lotes/ops])
    end

    J --> L(Tamiz o sistema que toma el pellet)
    L -- Retorno de Finos --> F
    L --> M[Tolvas de Producto Terminado]

    subgraph SALIDAS_FINALES [PUNTO 5: SALIDAS VENDIBLES]
        M --> N{Ruta de Salida}
        N --> O[ENSACADORA]
        O -- DATOS CLAVE --> DP5A([DATA POINT 5A Saco:<br>- Peso Promedio Real por Saco/ pesar aleatoriamente<br>- Conteo Total de Sacos])
        N --> P[CARGA A GRANEL]
        P -- DATOS CLAVE --> DP5B([DATA POINT 5B Granel:<br>- Peso Neto Báscula Salida])
    end

    DP2 -.-> CALCULO_SACKOFF
    DP5A -.-> CALCULO_SACKOFF
    DP5B -.-> CALCULO_SACKOFF
    DP3 -.-> ANALISIS_MERMA
    DP4 -.-> ANALISIS_MERMA

    subgraph RESULTADO [ANÁLISIS DE DATOS]
        CALCULO_SACKOFF[CÁLCULO SACK-OFF BRUTO<br>Entrada vs Salida Total]
        ANALISIS_MERMA[ANÁLISIS DE MERMA TÉCNICA<br>Balance de Humedad P2 vs P4]
    end

    %% Asignación de clases
    class A,C input;
    class D,E,F,G,I,J,L,M,N process;
    class O,P output;
    class DP2,DP3,DP4,DP5A,DP5B dataPoint;
    